In [2]:
import logging
logging.getLogger("matplotlib").setLevel(logging.INFO)
logging.getLogger("numba").setLevel(logging.INFO)
logging.getLogger("jieba").setLevel(logging.INFO)
import matplotlib.pyplot as plt
import IPython.display as ipd
from mel_processing import spectrogram_torch
import os

import torch
from torch.utils.data import DataLoader
from utils import load_wav_to_torch

import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaner import text_to_sequence

# logging.getLogger("matplotlib").setLevel(logging.INFO)
# logging.getLogger("matplotlib").setLevel(logging.INFO)
def get_text(text):
    text_norm = text_to_sequence(text)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
hps = utils.get_hparams_from_file("./configs/config.json")
net_g = SynthesizerTrn(
        len(symbols),
        hps.data.filter_length // 2 + 1,
        hps.data.hop_length,
        hps.data.sampling_rate,
        hps.train.segment_size // hps.data.hop_length,
        n_speakers=hps.data.n_speakers,
        **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("/Volumes/Extend/下载/G_220000.pth", net_g, None)

initialized zh frontend
load 
INFO:root:Loaded checkpoint '/Volumes/Extend/下载/G_220000.pth' (iteration 36)


In [21]:
def infer(text,spk, lang=""):
    sid = hps.data.spk2id[spk]
    stn_tst = get_text(lang+text+lang)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        sid = torch.LongTensor([sid])
        audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667)[0][0,0].data.cpu().float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [25]:
text1 = "。下面给大家简单介绍一下怎么使用这个教程吧！首先我们要有魔法，才能访问到谷歌的云平台。点击连接并更改运行时类型，设置硬件加速器为G P U。然后，我们再从头到尾挨个点击每个代码块的运行标志。可能需要等待一定的时间。当我们进行到语音合成部分时，就可以更改要说的文本，并设置保存的文件名啦。"
text2 = "。下面给大家简单介绍一下怎么使用这个教程吧！首先我们要有魔法，才能访问到谷歌的云平台。点击连接并更改运行时类型，设置硬件加速器为G P U。然后，我们再从头到尾挨个点击每个代码块的运行标志。并设置保存的文件名啦。"
# text2 = "这种逃亡有意义吗？"
# infer(text2, "biaobei")

### Voice Conversion

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))